In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import time

# 1. Prepare Data

## 1.1 Load

In [ ]:
%%time
train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
#display(train.info(), train.head())

# 2. Create Model

In [ ]:
def rmse(predict, actual):
    predict = np.array(predict)
    actual = np.array(actual)

    distance = predict - actual

    square_distance = distance ** 2

    mean_square_distance = square_distance.mean()

    score = np.sqrt(mean_square_distance)

    return score

rmse_score = make_scorer(rmse, greater_is_better = False)

In [ ]:
def model_training(train):
    features = [f'f_{i}' for i in range(300)]

    X = train[features][:10000]
    y = train['target'][:10000]
    del train
    X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.95, test_size=0.05,shuffle=True)
    pca = PCA(.98)
    scaler = StandardScaler()
    X_train=scaler.fit_transform(X_train)
    X_train=pca.fit_transform(X_train)
    X_train=pd.DataFrame(X_train) 
    
#     grid = GridSearchCV(SVR(),{'C': [10],'gamma': [1e-8],'epsilon':[0.001],'kernel': ['rbf']}, cv=10, return_train_score=False, verbose = 0,n_jobs=3,scoring=rmse_score)
#     grid.fit(X_train,y_train)
#     pp=grid.best_params_
#     print(pp)
    pp={'C': 1, 'gamma': 1e-8, 'kernel': 'rbf','epsilon':0.001}
    model=SVR()
    model.set_params(**pp)
    
    model=model.fit(X_train,y_train)

    return(model,pca,scaler,X_test,y_test)

In [ ]:
def get_prediction(data,model,pca,scaler):
    data=scaler.transform(data)
    data=pca.transform(data)
    data=pd.DataFrame(data)
    prediction=model.predict(data)
    return(prediction)
    

In [ ]:
model,pca,scaler,X_test,y_test=model_training(train)

In [ ]:
%%time
pred = get_prediction(X_test,model,pca,scaler)

pred

In [ ]:
# evaluate
pearsonr(y_test, pred)[0]

# 3. Submission

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    features = [f'f_{i}' for i in range(300)]
    test_df=test_df[features]
    sample_prediction_df['target'] = get_prediction(test_df,model,pca,scaler)  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions